# __Original notebook can be found here:__ https://www.kaggle.com/code/nandinitatiwala/traffic-sign-transfer-learning-classification

# Introduction
With automation becoming increasingly common, machine models being able understand traffic signs is key for self-driving cars and other forms of transport to ensure safety. As such, I wanted to incorporate deep learning into this and see how a machine learning model would respond to identifying various traffic signs.

The goal of this investigation is to see if, using a pertained model, I could get a model that is able to accurately classify traffic signs.

## Setting Up
First, we need to set up the general base for a machine learning model. 

In [ ]:
# loading general packages and libraries 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import os
import cv2
import random
from PIL import Image

# loading tensor flow libraries needed
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from keras.layers import Dropout, Flatten, MaxPool2D,BatchNormalization
from keras.preprocessing import image
from tensorflow.keras.optimizers import Adam,SGD
import keras
from keras.preprocessing import image

seed=1
np.random.seed(seed)
tf.random.set_seed(seed)

## Gather and Explore the Data
For this investigation, I will be using a dataset I found on Kaggle. The data consists of images of traffic signs which have been automatically registered so that the sign is centred and occupies around the same space in each image.

The are around 58 classes and each class has around 120 images. Since this dataset has not been cleaned and organised completely, we first need to ensure there is an adequete amount of photos in each class and that they are correctly categorised.

In [ ]:
# importing images
train_dir = "/kaggle/input/traffic-sign-dataset-classification/traffic_Data/DATA/" 

# reading the csv file
labels = pd.read_csv("/kaggle/input/traffic-sign-dataset-classification/labels.csv")
labels

In [ ]:
# no. of images per label
lst = []
for i in labels.index:
    lst.append(len(os.listdir(train_dir + str(i))))
labels['count'] = lst
labels['count'].describe()

In [ ]:
# only keep those with enough images in each label 
labels = labels[labels['count'] >= 107.5]
labels

In [ ]:
# finding the unknown image
fnames = os.listdir(train_dir + '56')
img = cv2.imread(train_dir + '56/' + fnames[3])
plt.imshow(img)

In [ ]:
# renaming
labels.loc[56, "Name"] = "Yield"

labels["Name"] = ["Speed Limit 5", "Speed Limit 40",
       "Speed Limit 60", "Speed Limit 80", "No Left",
       "No Overtake from Left", "No Cars", "No Horn", "Keep Right",
       "Watch for Cars", "Bicycle Crossing", "Zebra Crossing",
       "No Stopping", "No Entry", "Yield"]

Now that we have cleaned the data, we can load the libraries.

In [ ]:
# set the image size 
image_size = 128

# input and data augmentation
train_datagen = ImageDataGenerator(
        rescale = 1 / 255.,
        rotation_range=10,
        zoom_range=0.2,
        width_shift_range=0.1,
        height_shift_range=0.1,                                       
        fill_mode="nearest",
        validation_split=0.25,
    )

train_generator = train_datagen.flow_from_directory(
    directory = train_dir,          
    target_size = (image_size, image_size), 
    batch_size = 28,
    shuffle=True,
    class_mode = "categorical",   
    subset = "training"     
)

validation_generator = train_datagen.flow_from_directory(
    directory = train_dir,   
    target_size = (image_size, image_size),   
    batch_size = 28, 
    class_mode = "categorical",
    subset = "validation"
)

## Transfer Learning

Since the data is ready, it is time to train the model using the pretrained ResNet50 model. This is already done through transfer learning, which takes a pretrained model, removes the final layer, and replaces that last layer with the relevant output.

ResNet50 is quite a popular model and accurate for this project since it is commonly used for traffic sign analysis.

Below, is the first model (removing the final layer of the ResNet50 model and replacing it with a Dense final layer with the number of nodes being the number of outputs).

### Model 1

In [ ]:
# set classes to number of categories and input weight paths
num_classes = 58
resnet_weights_path = '../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

# defining the model
model = Sequential()
model.trainable = True

model.add(ResNet50(include_top=False, pooling='avg', weights= resnet_weights_path))
model.add(BatchNormalization())
model.add(Dense(48, activation='relu'))
model.add(Dropout(0.25))
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

Now, we need to compile the final layer of the network using categorical_crossentropy as the loss function and the stochastic gradient descent (as the optimizer) to minimize the categorical cross-entropy loss. 

In [ ]:
model.compile(optimizer="sgd", loss='categorical_crossentropy', metrics=['accuracy'])

Now we fit the model for training and validation using 12 epochs and 60 steps per epoch.

In [ ]:
history = model.fit(
        train_generator,
        steps_per_epoch= 60,
        epochs = 12,
        validation_data=validation_generator,
        validation_steps=5)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

This wasn't bad at all for a first model! We got a accuracy score of 88% and a validation accuracy of 52%. Let's see if it can be improved with a different optimiser called adam and changing a few parameters.

### Model 2

In [ ]:
# define the model (add layers)
model2 = Sequential()
model2.trainable = True

model2.add(ResNet50(include_top=False, pooling='avg', weights= resnet_weights_path))
model2.add(BatchNormalization())
model2.add(Dense(48, activation='relu'))
model2.add(BatchNormalization())
model2.add(Dense(256, activation='relu'))
model2.add(Dropout(0.25))
model2.add(Dense(128, activation='relu'))
model2.add(Dense(num_classes, activation='softmax'))

model2.summary()

In [ ]:
model2.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history2 = model2.fit(
        train_generator,
        steps_per_epoch= 60,
        epochs = 12,
        validation_data=validation_generator,
        validation_steps=5)

In [ ]:
plt.plot(history2.history['accuracy'])
plt.plot(history2.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

This model is slightly better! With an accuracy score of 89%, this was improved however the validation accuracy was only 10%. Using this as the base, lets tune the parameters to see if we can improve the validation accuracy score.

### Model 3

In [ ]:
# define the model
model3 = Sequential()
model3.trainable = True

model3.add(ResNet50(include_top=False, pooling='avg', weights= resnet_weights_path))
model3.add(BatchNormalization())
model3.add(Dense(48, activation='relu'))
model3.add(BatchNormalization())
model3.add(Dense(256, activation='relu'))
model3.add(Dropout(0.25))
model3.add(Dense(128, activation='relu'))
model3.add(Dense(num_classes, activation='softmax'))

model3.summary()

In [ ]:
model3.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# tuning to try to improve the score
history3 = model3.fit(
        train_generator,
        steps_per_epoch= 60,
        epochs = 15,
        validation_data=validation_generator,
        validation_steps=7)

In [ ]:
plt.plot(history3.history['accuracy'])
plt.plot(history3.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

This was much better! It seems Model 3 resulted in the best scores with an accuracy score of 92% and a validation score of 68%.

## Prediction
Let's use an image from the testing data to see the accuracy of the model.

In [ ]:
label = {0:"Speed Limit 5", 1:"Speed Limit 15", 2:"Speed Limit 30", 
         3:"Speed Limit 40", 4:"Speed Limit 50", 5:"Speed Limit 60", 
         6:"Speed Limit 70", 7:"Speed Limit 80", 8:"Don't go straight or left", 
         9:"Don't go straight or right", 10:"Don't go straight", 11:"No Left",
         12:"Don't go right or left", 13:"Don't go right", 14:"No Overtake from Left", 
         15:"No U-turn", 16:"No Cars", 17:"No Horn", 18:"Speed Limit (40km/h)",
         19:"Speed Limit (50km/h)", 20:"Go straight or right", 21:"Watch out for cars",
         22:"Go left", 23:"Go left or right", 24:"Go right", 25:"Keep Left",
         26:"Keep Right", 27:"Roundabout mandatory", 28:"Go Straight",
         29:"Horn", 30:"Bicycle Crossing", 31:"U-turn", 32:"Road Divider",
         33:"Traffic Signals", 34:"Danger ahead", 35:"Zebra Crossing",
         36:"Bicycle Crossing", 37:"Children Crossing", 38:"Dangerous curve to the left",
         39:"Dangerous curve to the right", 40:"Unknown 1", 41:"Unknown 2", 42:"Unknown 3",
         43:"Go right or straight", 44:"Go left or straight", 45:"Unknown 4", 
         46:"Zigzag curve", 47:"Train Crossing", 48:"Under construction", 49:"Unknown 5",
         50:"Fences", 51:"Heavy Vehicle Accidents", 52:"Unknown 6", 53:"Give way",
         54:"No Stopping", 55:"No Entry", 56:"Yield", 57:"Unknown 8"}

img_directory = "/kaggle/input/traffic-sign-dataset-classification/traffic_Data/TEST/021_1_0008.png"
test_image = image.load_img(img_directory, target_size=(128, 128))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
test_image

result = model3.predict(test_image)
    
img = mpimg.imread(img_directory)
imgplot = plt.imshow(img)
plt.show()
    
print(f"Predicted class: {label[np.argmax(result)]}")

As you can see, the predicticted class matches what the image is saying showing that our model works well!

## Conclusion
Through this investigation, I feel that we were able to be pretty successful in my goal. I found a model using a pretrained one that was able to fairly accurately predict and determine common traffic signs.

However, I think the data I used was quite standardised, so I'm not sure if this accuracy would be the same for photos where there is more background noise or it is further away. A major challenge was trying to make sure that the model was not overfit to the data as certain classes did not have many images and some images were in mutiple classes as well. 

Nonetheless, I think this model is very applicable to the real world with further steps. It can be used for automation in transport or to help the visually impaired with road safety.

## Future Work
What are the next steps?

1. Having live image detection so cars can respond in real time
2. Converting it to speech for audio aid
3. Adding a wider variety of road signs / altering based on country

## References
1. Transfer learning example notebook (goes with the video demonstration)- https://www.kaggle.com/dansbecker/transfer-learning/
2. Kaggle Computer Vision course final notebook on Data Augmentation- https://www.kaggle.com/ryanholbrook/data-augmentation
3. Traffic Sign Dataset
https://www.kaggle.com/datasets/ahemateja19bec1025/traffic-sign-dataset-classification